# Dry Run with Sample File - Colab

## Functions

In [ ]:
def print_shape(df):
    print(f"Rows: {df.shape[0]:,} \nColumns: {df.shape[1]:,}")

In [ ]:
def keep_latest(df, dedupe_by_cols, sort_by_cols):
    """Keep last record for each unique combination of dupe_cols, ordering by sort_cols"""
    df = df.sort_values(sort_by_cols).drop_duplicates(dedupe_by_cols, keep="last")
    return df

In [ ]:
# calculate percent missing for each field in df
def percent_missing(df):
    # Total missing values
    mis_val = df.isnull().sum()
    # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(
        columns={0: "Missing Values", 1: "% of Total Values"}
    )
    # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = (
        mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:, 1] != 0]
        .sort_values("% of Total Values", ascending=False)
        .round(1)
    )
    # Print some summary information
    print(
        "Your selected dataframe has " + str(df.shape[1]) + " columns.\n"
        "There are "
        + str(mis_val_table_ren_columns.shape[0])
        + " columns that have missing values."
    )
    # Return the dataframe with missing information
    return mis_val_table_ren_columns


## Environment

In [ ]:
! pip install fastparquet

shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/cli/main.py", line 9, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/cli/main_parser.py", line 8, in <module>
    from pip._internal.cli import cmdoptions
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/cli/cmdoptions.py", line 23, in <module>
    from pip._internal.cli.parser import C

In [ ]:
import os
import pandas as pd
import pyarrow as pa
import fastparquet as fp
import datetime

In [ ]:
# mount google drive
# note only used for merge
from google.colab import drive
import os
drive.mount('/gdrive')

# change dir
cur_path = '/gdrive/MyDrive/capstone/'
os.chdir(cur_path)
!pwd

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/School/Capstone


## Data Composition
Combine and standardize data

### Listings

#### Column Selection

In [ ]:
load_cols = [
    "vin",
    "status_date",
    "price",
    "miles",
    "year",
    "make",
    "model",
    "trim",
    "vehicle_type",
    "body_type",
    "body_subtype",
    "drivetrain",
    "fuel_type",
    "engine_block",
    "engine_size",
    "transmission",
    "doors",
    "cylinders",
    "city_mpg",
    "highway_mpg",
    "base_exterior_color",
    "base_interior_color",
    "is_certified",
    "state",
    "zip",
    "latitude",
    "longitude",
    "dealer_type",
    "photo_links_count",
    "photo_main",
    "listed_options",
    "hvf_options",
    "seller_comments",
]


In [ ]:
excluded = [
    "id", # unique id for marketcheck database
    "heading", # title of listing
    "msrp", # unsure
    "stock_no", # stock number marketcheck/dealer
    "interior_color", # full description, not standardized
    "exterior_color", # full description, not standardized
    "is_transfer", # 85% na
    "taxonomy_vin", # might be useful later on to group vehicles
    "scraped_at",
    "first_scraped_at",
    "source", # website
    "seller_name",
    "city",
    "car_seller_name",
    "car_city",
    "car_state",
    "car_zip",
    "car_latitude",
    "car_longitude",
    "dom",
    "dom_180",
    "dom_active",
    "currency_indicator", # all usd
    "miles_indicator", # all miles
    "carfax_1_owner",
    "carfax_clean_title",
    "loan_term", # 86% na
    "loan_apr", # 87% na
    "l_down_pay", # all na
    "l_emi", # all na
    "f_down_pay", # 93% na
    "f_down_pay_per", # 95% na
    "f_emi", # 94% na
    "lease_term", # all na
    "engine_measure", # all na
    "engine_aspiration", # all na
    "speeds", # all na
    "engine" # redundant with engine_block, engine_size, cylinders
    ]


#### The Rest

In [ ]:
os.listdir()

['Files',
 'meeting_recordings',
 'Archive',
 'Proposal.docx',
 'Final Report.gdoc',
 'large_sample',
 'oh_mvr_out.parquet',
 'tn_mvr.parquet',
 'tx_mvr_out.parquet',
 'Final Presentation.gslides',
 'merged.parquet',
 '8_colab.ipynb']

In [ ]:
# dir path
parquet_dir = "large_sample"

# construct list of parquet files
parquet_files = [
    os.path.join(parquet_dir, f)
    for f in os.listdir(parquet_dir)
    if f.endswith(".parquet")
]

# read into pandas dataframe
listings_df = pd.concat([pd.read_parquet(f, columns=load_cols) for f in parquet_files])
print_shape(listings_df)


Rows: 3,353,363 
Columns: 33


In [ ]:
pd.set_option("display.max_rows", 100)
listings_df.sample(5).T

,7661,1657,10806,1753,8443
vin,1G1ZB5ST7KF223859,JTDKARFU5J3549164,1C4BJWFG3GL344024,1C3CCCAB5GN133828,KNDJT2A51D7580292
status_date,1625727807,1659729461,1661980581,1620047208,1630327352
price,24989.0,28755.0,34995.0,12700.0,NaN
miles,31553.0,99123.0,59044.0,48720.0,92493.0
year,2019.0,2018.0,2016.0,2016.0,2013.0
make,Chevrolet,Toyota,Jeep,Chrysler,Kia
model,Malibu,Prius,Wrangler Unlimited,200,Soul
trim,1LS,Four,Rubicon,Limited,Base
vehicle_type,Car,Car,Truck,Car,Car
body_type,Sedan,Hatchback,SUV,Sedan,Hatchback


In [ ]:
# convert to date
listings_df["status_date"] = (
    listings_df["status_date"].fillna(0).map(lambda x: datetime.datetime.fromtimestamp(int(x)))
)

In [ ]:
# dedupe listings by vin, keeping latest status_date
listings_df = listings_df.sort_values("status_date").drop_duplicates("vin", keep="last")
print_shape(listings_df)


Rows: 2,333,582 
Columns: 33


In [ ]:
percent_missing(listings_df)

Your selected dataframe has 33 columns.
There are 28 columns that have missing values.


,Missing Values,% of Total Values
body_subtype,1895089,81.2
base_interior_color,548807,23.5
is_certified,532850,22.8
seller_comments,523752,22.4
price,322040,13.8
dealer_type,288796,12.4
base_exterior_color,212545,9.1
engine_block,168285,7.2
cylinders,168284,7.2
engine_size,166983,7.2


In [ ]:
# if body_subtype is not null, append to body_type
# subtypes are specific to trucks
listings_df["body_type"] = listings_df["body_type"].fillna("") + " " + listings_df["body_subtype"].fillna("")

# strip whitespace and replace empty strings with None
listings_df["body_type"] = listings_df["body_type"].str.strip().replace("", None)

In [ ]:
# drop body_subtype
listings_df = listings_df.drop("body_subtype", axis=1)

### Registrations

#### Texas

In [ ]:
tx_cols = ['VIN', 'SALE_DATE', 'SALES_PRICE', 'ODOMETER_BRAND', 'ODOMETER_READING',
       'DOCNO', 'VEHYEAR', 'MAKE', 'MODEL', 'BODY_TYPE', 'OWNERSHIP_CD',
       'TTL_SIGNED_DATE', 'OWNER_ZIP', 'OWNER_ZIP+4', 'RENEW_ZIP',
       'RENEW_ZIP+4', 'RECONDITION_CD', 'SALVSTATECNTRY']

In [ ]:
tx_file = "tx_mvr_out.parquet"

registrations_tx = fp.ParquetFile(tx_file).to_pandas(
    columns=["VIN", "SALE_DATE", "SALES_PRICE"]
)

print_shape(registrations_tx)


Rows: 29,053,057 
Columns: 3


In [ ]:
# rename columns
registrations_tx = registrations_tx.rename(columns={'VIN':'vin','SALE_DATE':'mvr_purchase_date','SALES_PRICE':'mvr_price'})

In [ ]:
# convert to date type
registrations_tx['mvr_purchase_date'] = pd.to_datetime(registrations_tx['mvr_purchase_date'],format="\'%Y-%m-%d\'")

In [ ]:
# dedupe vin by date
registrations_tx = keep_latest(registrations_tx, ["vin"], ["mvr_purchase_date"])
print_shape(registrations_tx)

Rows: 22,726,694 
Columns: 3


In [ ]:
registrations_tx.dtypes

vin                          object
mvr_purchase_date    datetime64[ns]
mvr_price                   float32
dtype: object

#### Ohio

In [ ]:
oh_cols = ['TitleNumber', 'IssueDate', 'TitleStatus', 'TitleType', 'VIN', 'Year',
       'Make', 'Model', 'BodyType', 'NewOrUsed', 'Mileage', 'MileageBrand',
       'MileageJustificationType', 'NMVTISBrand1', 'NMVTISBrand2',
       'NMVTISBrand3', 'PurchaseDate', 'PurchasePrice', 'OwnerType',
       'OwnerName', 'PrevOwnerType', 'PrevOwnerName', 'CountyofResidence',
       'PreviousStateCode']

In [ ]:
oh_file = "oh_mvr_out.parquet"

registrations_oh = fp.ParquetFile(oh_file).to_pandas(
    columns=['VIN','PurchaseDate','PurchasePrice']
)

print_shape(registrations_oh)

Rows: 28,591,523 
Columns: 3


In [ ]:
# rename columns
registrations_oh = registrations_oh.rename(columns={'VIN':'vin','PurchaseDate':'mvr_purchase_date','PurchasePrice':'mvr_price'})

In [ ]:
registrations_oh.sample(5)

,vin,mvr_purchase_date,mvr_price
22186803,5NPDH4AE4DH360764,2022-03-31,3000.0
11586438,1V2UR2CA6KC516967,2022-11-07,0.0
10078721,3GNKBCR43MS565844,2021-06-10,36478.0
6685893,2G2WP552X81189936,2019-05-23,0.0
2619716,KMHCT4AE3HU268658,2017-12-26,17358.0


In [ ]:
# convert to date type
registrations_oh['mvr_purchase_date'] = pd.to_datetime(registrations_oh['mvr_purchase_date'],format="%Y-%m-%d")

In [ ]:
# dedupe vin by date
registrations_oh = keep_latest(registrations_oh, ["vin"], ["mvr_purchase_date"])
print_shape(registrations_oh)

Rows: 13,422,774 
Columns: 3


In [ ]:
registrations_oh.dtypes

vin                          object
mvr_purchase_date    datetime64[ns]
mvr_price                   float32
dtype: object

#### Tennessee

In [ ]:
tn_cols = ['vin', 'price', 'odometer_type', 'mileage', 'county', 'zip',
       'model_year', 'make', 'model', 'vehicle_type', 'new_used',
       'title_issue_date', 'purchase_date']

In [ ]:
tn_file = "tn_mvr.parquet"

registrations_tn = fp.ParquetFile(tn_file).to_pandas(
    columns=['vin','purchase_date','price']
)

print_shape(registrations_tn)

Rows: 9,831,774 
Columns: 3


In [ ]:
# rename columns
registrations_tn = registrations_tn.rename(columns={'vin':'vin','purchase_date':'mvr_purchase_date','price':'mvr_price'})

In [ ]:
registrations_tn.head(5)

,vin,mvr_purchase_date,mvr_price
0,\1FTRX07L53KD87737,2018-06-04,0.0
1,0000000V464048305,2020-05-13,0.0
2,0000161231,2021-12-08,1800.0
3,0005400FKXX,2019-03-04,0.0
4,0096456,2022-11-15,300.0


In [ ]:
# convert to date type
registrations_tn['mvr_purchase_date'] = pd.to_datetime(registrations_tn['mvr_purchase_date'],format="%Y-%m-%d", errors="coerce")

In [ ]:
# dedupe vin by date
registrations_tn = keep_latest(registrations_tn, ["vin"], ["mvr_purchase_date"])
print_shape(registrations_tn)

Rows: 7,558,173 
Columns: 3


In [ ]:
registrations_tn.dtypes

vin                          object
mvr_purchase_date    datetime64[ns]
mvr_price                   float32
dtype: object

### Join

In [ ]:
print_shape(listings_df)

Rows: 2,333,582 
Columns: 32


In [ ]:
# stack all registrations
registrations = pd.concat([registrations_tx, registrations_oh, registrations_tn])

In [ ]:
print_shape(registrations)

Rows: 43,707,641 
Columns: 3


In [ ]:
df = listings_df.merge(registrations, on='vin', how='inner')
print_shape(df)

Rows: 1,858,778 
Columns: 34


In [ ]:
# match rate
round(df["vin"].nunique()/listings_df["vin"].nunique(),2)

0.78

## Split HVF

In [ ]:
df["hvf_options_standard"] = df["hvf_options"].map(lambda x: x[0])

In [ ]:
df["hvf_options_optional"] = df["hvf_options"].map(lambda x: x[1])

In [ ]:
df = df.drop("hvf_options", axis=1)

## Write

In [ ]:
# save merged file
df.to_parquet("merged.parquet", index=True)

In [ ]:
# write unstructured columns to parquet (listed_options, hvf_options, seller_comments)
df[["listed_options", "hvf_options_standard", "hvf_options_optional", "seller_comments"]].to_parquet(
    "large_sample-unstructured_features.parquet", index=True
)


In [ ]:
# write to parquet, excluding unstructured features
df.drop(["listed_options", "hvf_options_standard", "hvf_options_optional", "seller_comments"], axis=1).to_parquet(
    "large_sample-standard_features.parquet", index=True
)


## Data Reload

In [ ]:
# whole file
merged_file = "merged.parquet"
df = fp.ParquetFile(merged_file).to_pandas()
print_shape(df)

Rows: 1,858,778 
Columns: 35


In [ ]:
# unstructured features only
unstructured_file = "large_sample-unstructured_features.parquet"
df = fp.ParquetFile(unstructured_file).to_pandas()
print_shape(df)

Rows: 1,858,778 
Columns: 4


In [ ]:
# standard features only
standard_file = "large_sample-standard_features.parquet"
df = fp.ParquetFile(standard_file).to_pandas()
print_shape(df)

## High Value Options

In [ ]:
# count of high value feature options
flat_list = list(itertools.chain(*df["hvf_options"]))
distinct_elements = set(itertools.chain(*flat_list))
print(len(distinct_elements))
distinct_elements

82

In [ ]:
# select
merged_file = "merged.parquet"
df = fp.ParquetFile(merged_file).to_pandas(columns = ["hvf_options_standard", "hvf_options_optional"])
print_shape(df)

Rows: 1,858,778 
Columns: 2


In [ ]:
hvf_options = df["hvf_options_standard"] + df["hvf_options_optional"]

In [ ]:
hvf_options = hvf_options.apply(set).apply(list)

In [ ]:
hvf_options_df = pd.get_dummies(hvf_options.apply(pd.Series).stack()).groupby(level=0).sum()
hvf_options_df.head()

<ipython-input-16-93b04dfc525d>:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  hvf_options_df = pd.get_dummies(hvf_options.apply(pd.Series).stack()).groupby(level=0).sum()


,3rd Row Seats,4-Wheel Steering,Adaptive Cruise Control,Android Auto,Anti Collision System,Apple CarPlay,Automatic Transmission,Autonomous Drive Functions,Aux Jack Input,Backup Camera,...,Touch Screen Audio,Trailer Assist,Trailer Tow Mirrors,Turbo Boost,USB Connection,Video Entertainment,Voice Recognision,Voice Recognition,WiFi Hotspot,Wireless Charging/Connection
3,1,0,0,0,0,0,1,0,1,1,...,0,0,0,0,1,0,1,0,0,0
7,1,0,0,0,0,0,1,0,1,0,...,0,1,0,0,0,0,0,1,0,0
10,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,1,1,0,1,0,0
11,1,0,0,0,0,1,1,0,1,0,...,1,1,0,0,1,0,0,1,1,0
12,0,0,0,0,0,0,1,0,1,1,...,1,0,0,0,1,0,0,1,0,0


In [ ]:
hvf_options_df.shape

(1585169, 82)

In [ ]:
# save file
hvf_options_df.to_parquet("hvf_options.parquet", index=True)

## Listed Options

In [ ]:
import itertools

In [ ]:
pd.set_option("display.max_colwidth", 1000)
df[["listed_options"]].sample(5)

,listed_options
1208407,"[Alloy wheels, Front reading lights, Outside temperature display, SYNC, Tachometer, Front beverage holders, Satellite Radio Ready, Overhead console, SERVICE RECORDS AVAILABLE, Variably intermittent wipers, Rear window wiper, Parking Sensors / Assist, Illuminated entry, Low tire pressure warning, IPOD PLUGIN, Axle Ratio: 3.16, Panic alarm, CD player, Power windows, Rear anti-roll bar, Power passenger seat, Safety Check by Ford Certified Technicians, Power driver seat, Air Conditioning, Delay-off headlights, Compass, Bluetooth, Rear reading lights, Backup Camera, Emergency communication system, BACKUP CAMERA, Remote keyless entry, Knee airbag, 6 Speakers, Bumpers: body-color, Wheels: 18"" Aluminum Painted Sparkle Silver, Leather Shift Knob, Rear seat center armrest, Power Seats, Speed-Sensitive Wipers, Power door mirrors, Dual front impact airbags, Passenger door bin, SATELLITE RADIO, Driver door bin, Dual front side impact airbags, Traction control, Auto-dimming Rear-View mirror, Non..."
2487,[None]
647909,"[Rear shoulder room: 1,135mm (44.7""), Sequential multi-point fuel injection, Outside temperature display, Front shoulder room: 1,417mm (55.8""), Exterior length: 4,161mm (163.8""), Tachometer, Front beverage holders, Fuel economy highway: 21mpg, Front legroom: 1,041mm (41.0""), Variably intermittent wipers, Exterior body width: 1,872mm (73.7""), Ground clearance (max): 231mm (9.1""), Low tire pressure warning, Tires: all-terrain, CD player, Front fog lights, Rear anti-roll bar, Front hiproom: 1,412mm (55.6""), Rear cargo: conventional, Ramp breakover angle: 22 deg, Compression ratio: 10.20 to 1, Payload: 454kg (1,000lbs), Number of valves: 24, Ignition disable, Compass, Drive type: four-wheel, Variable valve control, Interior cargo volume: 362 L (13 cu.ft.), Fuel tank capacity: 18.6gal., Electronic stability, Integrated roll-over protection, Departure angle: 37 deg, Max seating capacity: 4, Fuel economy combined: 18mpg, Passenger volume: 2,503L (88.4 cu.ft.), Approach angle: 41 deg, GVWR..."
657230,"[Front shoulder room: 1,647mm (64.8""), Sequential multi-point fuel injection, Alloy wheels, Limited slip differential, Front reading lights, Outside temperature display, Towing capacity: 5,897kg (13,000lbs), Tachometer, Bedliner, Front beverage holders, Overhead console, Right rear passenger: conventional, Variably intermittent wipers, Bumpers: chrome, Door mirrors: body-color, Front hiproom: 1,543mm (60.7""), Rear door bins, Illuminated entry, Bodyside moldings, Rear headroom: 983mm (38.7""), Low tire pressure warning, Panic alarm, Tires: all-terrain, Power windows, CD player, Front fog lights, Transmission: 6 speed automatic, Parking sensors: rear, Payload: 1,415kg (3,120lbs), Voltmeter, Power driver seat, Ignition disable, Delay-off headlights, Perimeter/approach lights, Compass, Passenger cancellable airbag, Rear reading lights, Drive type: four-wheel, 1-touch down, Turning radius: 7.5m (24.6'), Emergency communication system, Variable valve control, Remote keyless entry, Exterio..."
1358420,"[AM/FM radio: XM, Front reading lights, Moldings, bodyside, body-colored (Moldings are deleted if any SEO paint is ordered.), SL Decor, TRANSMISSION, 4-SPEED AUTOMATIC, ELECTRONICALLY CONTROLLED with overdrive and tow/haul mode (STD) (On Crew Cab models, requires (L20) Vortec 4.8L V8 SFI Flex-Fuel engine. Not available on T*10953 models.) (M30), Instrumentation, analog with speedometer, fuel level, engine temperature, and tachometer, Tire Pressure Monitor System (does not apply to spare tire), Windows, power with driver Express-Down, Grille, chrome surround (Not included when (VAT) Chrome grille, LPO is ordered.), Tachometer, Tires, P245/70R17 all-season, blackwall includes a blackwall spare tire (Requires 2WD models.), Heavy-Duty Auxiliary External Transmission Oil Cooler, Bumper, rear chrome, step-style with pad, Front beverage holders, Suspension, front independent, coil over shock, 17"" x 7.5"" 6-Lug Chrome

In [ ]:
type(df.loc[1208407,"listed_options"])

list

In [ ]:
# Flatten the list of lists into a single list
flat_list2 = list(itertools.chain(*df["listed_options"]))

# Convert the list to a set to get distinct elements
#distinct_elements2 = set(itertools.chain(*flat_list2))

test = list(set(flat_list2))

In [ ]:
test

['',
 'WHEELS 20" X 9" (50.8 CM X 22.9 CM) POLISHED-ALUMINUM (STD)',
 'ETR AM/FM Stereo w/CD/RDS',
 '[N10] ILLUMINATED KICK PLATES $465',
 'Premium DIS Navigation with 9.2" High Def. Displa',
 'Cargo: Liftgate, AM/FM Radio: SiriusXM, Remote Keyless Entry, Exterior Parking Camera Rear, Cruise Control, Rear Air Conditioning, Rear Seat Center Armrest, Unique Cloth Bucket Seats, 6 Speakers, Power Steering, Steering Wheel Mounted Audio Controls, Telescoping Steering Wheel, Speed-sensing Steering, Front Bucket Seats, 3rd Row Seats: Split-bench, Front Center Armrest, Telescopic Steering Column, Leather Steering Wheel, Tilt Steering Wheel, Delay-off Headlights, Fully Automatic Headlights, Bumpers: Body-color, Heated Door Mirrors, Variably Intermittent Wipers, Low Tire Pressure Warning, Speed-Sensitive Wipers, Power Windows',
 'Leather-wrapped Park Brake Handle',
 'Lighting-inc: overhead courtesy, map, cargo area',
 'Trailer Towing - Class II with receiver hitch - good up to 3500 and comes with

In [ ]:
distinct_elements2

{' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '{',
 '}',
 '~'}